In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
from scipy import ndimage
import os
from skimage.filters import threshold_otsu
from skimage.segmentation import slic, mark_boundaries

## FUNCTIONS

#### Sobel Filter implementation

In [2]:
def sobel_filter(image):
    #added dtype float64 to get a better result
    x_kernel = np.array([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]], 
                        dtype=np.float64)
    
    y_kernel = np.array([[-1, -2, -1],
                        [0, 0, 0],
                        [1, 2, 1]], 
                        dtype=np.float64)
    
    image = image.astype(np.float64)
    
    x_gradient = ndimage.convolve(image, x_kernel)
    y_gradient = ndimage.convolve(image, y_kernel)
    
    edges = np.sqrt(x_gradient**2 + y_gradient**2)
    edges = edges / edges.max() * 255

    theta_matrix  = np.arctan2(y_gradient, x_gradient)
    
    return edges, theta_matrix

In [3]:
def non_max_suppression(magnitude, direction):
    height, width = magnitude.shape
    non_max_matrix = np.zeros((height,width), dtype=np.int32)
    angle = direction * 180. / np.pi
    angle[angle < 0] += 180

    
    for i in range(1,height-1):
        for j in range(1,width-1):
            q = 255
            r = 255

           #angle 0
            if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                q = magnitude[i, j+1]
                r = magnitude[i, j-1]
            #angle 45
            elif (22.5 <= angle[i,j] < 67.5):
                q = magnitude[i+1, j-1]
                r = magnitude[i-1, j+1]
            #angle 90
            elif (67.5 <= angle[i,j] < 112.5):
                q = magnitude[i+1, j]
                r = magnitude[i-1, j]
            #angle 135
            elif (112.5 <= angle[i,j] < 157.5):
                q = magnitude[i-1, j-1]
                r = magnitude[i+1, j+1]

            if (magnitude[i,j] >= q) and (magnitude[i,j] >= r):
                non_max_matrix[i,j] = magnitude[i,j]
            else:
                non_max_matrix[i,j] = 0

    
    return non_max_matrix

#### Canny Edge implementation

In [4]:
def canny_edge_detection(image,low_threshold=40, high_threshold=80):
    height, width = image.shape
    
    blurred_image = cv2.GaussianBlur(image, (5, 5), 0)
    magnitude, directions = sobel_filter(blurred_image)
    non_max = non_max_suppression(magnitude, directions)
    
    weak_value = 25
    strong_value = 255
    
    link_matrix = non_max.copy()
    link_matrix = np.where(non_max > high_threshold, strong_value, np.where(non_max > low_threshold, weak_value,0))

    for i in range(1, height-1):
        for j in range(1, width-1):
            if(link_matrix[i,j] == weak_value):
                if ((link_matrix[i+1, j-1] == strong_value) or 
                    (link_matrix[i+1, j] == strong_value) or 
                    (link_matrix[i+1, j+1] == strong_value) or 
                    (link_matrix[i, j-1] == strong_value) or 
                    (link_matrix[i, j+1] == strong_value) or 
                    (link_matrix[i-1, j-1] == strong_value) or 
                    (link_matrix[i-1, j] == strong_value) or 
                    (link_matrix[i-1, j+1] == strong_value)):
                    link_matrix[i, j] = strong_value
                else:
                    link_matrix[i, j] = 0

    return link_matrix

#### Hough Transform implementation

In [5]:
def hough_lines(edge_image, edge_threshold=200, line_threshold_min=100, theta_min=0, theta_max=180, num_of_lines=0, desc=True):
    
    # initializing accumulator space
    # defining the ranges for parameters
    rho_max = round(np.sqrt(edge_image.shape[1]**2 + edge_image.shape[0]**2)) # max distance from origin 
    rhos_size = 2 * rho_max + 1
    # creating an array to store and iterate in thetas and rhos
    thetas = np.deg2rad(np.arange(theta_min, theta_max))
    rhos = np.arange(-rho_max, rho_max+1)
    accumulator = np.zeros((rhos.shape[0],thetas.shape[0]))
    
    width, height = edge_image.shape
    
    # filling accumulator array
    for w in range(width):
        for h in range(height):
            if edge_image[w][h] > edge_threshold:
                for t in range(len(thetas)):
                    cos_theta = np.cos(thetas[t])
                    sin_theta = np.sin(thetas[t])
                    rho = w * cos_theta + h * sin_theta
                    for r in range(len(rhos)):
                        if rhos[r] > rho:
                            break    
                    accumulator[r][t] += 1 

    # padding the array for easier computation of local maximas
    padded_array = cv2.copyMakeBorder(accumulator, 1, 1, 1, 1, cv2.BORDER_CONSTANT, value=0) 
    local_maximas = []
    # iterate through each element in the 2D array
    for r in range(1, len(rhos)+1):
        for t in range(1, len(thetas+1)):
            if  padded_array[r,t] > line_threshold_min \
                and padded_array[r, t] > padded_array[r - 1, t] and padded_array[r, t] > padded_array[r + 1, t] \
                and padded_array[r, t] > padded_array[r, t - 1] and padded_array[r, t] > padded_array[r, t + 1]:
                local_maximas.append((rhos[r-1], thetas[t-1], accumulator[r-1,t-1]))
    # sorting the local maximas according to votes
    sorted_maximas = sorted(local_maximas, key=lambda x: x[2],reverse=desc)
    if num_of_lines != 0:
        return sorted_maximas[:num_of_lines]
    else:
        return sorted_maximas

In [6]:
def hough_to_cartesian(rho, theta, weight = 500):
    rho = np.float32(rho)
    theta = np.float32(theta)
    a = np.sin(theta)
    b = np.cos(theta)
    x0 = a * rho
    y0 = b * rho
    pt1 = (int(x0 + weight*(-b)), int(y0 + weight*(a)))
    pt2 = (int(x0 - weight*(-b)), int(y0 - weight*(a)))
    return pt1, pt2

In [7]:
def draw_hough_lines(lines, image):
    for line in lines:
        pt1, pt2 = hough_to_cartesian(line[0], line[1])
        cv2.line(image, pt1, pt2, (0,0,255), 2, cv2.LINE_AA)
    return image

## Detecting squares from hough lines
After implementing every function needed for hough transformation to detect squares from these hough lines i implemented an approach similar to the article **Rectangle Detection based on a Windowed Hough Transform**. But since my hough algorithm was not elegant as them the square detecting algorithm of mine is a simplified version. I don't create a window and look all of the image. 
The mathematics are still more or less same except for some +- signs i needed to change because of my algorithm implementation.

I find horizontal and vertical hough lines seperatly since in most of the images only horizontal lines makes the certain threshold and assigned as lines. After that i try to find pairs that are paralel by these conditions:
$$ ∆θ = |θi − θj | < Tθ $$
$$ ∆ρ = |ρi + ρj | < Tρ $$ 
If they met these conditions then i find the peak these lines create with formula:
$$ αk = (θi + θj)/2 $$ 
$$ ξk = |ρi − ρj |/2. $$
After that i to match vertical and horizontal peaks that are orthogonal with some threshold. A rectangle is then detected if:
$$ ∆α = ||αk − αl| − 90◦| < Tα $$
Because there are a lot of rectangles these lines can create i find the rectangle that is more suitible with the formula:

$$
E(P_k, P_l) = \sqrt{a (\Delta\theta_k^2 + \Delta\theta_l^2 + \Delta\alpha^2) + b (\Delta\rho_k^2 + \Delta\rho_l^2)}
$$
Then i get the rectangle with least error rate. 
Note: all formulas are taken from the article

In [8]:
def find_pairs(lines, threshold_rho = 20, threshold_theta = 0.15):
    pairs = []
    for i in range(len(lines)-1):
        for j in range(i+1,len(lines)):
            if np.abs(lines[i][0] - lines[j][0]) > threshold_rho \
            and np.abs(lines[i][1] - lines[j][1]) < threshold_theta:
                pairs.append([lines[i][:2],lines[j][:2]])
    return pairs

In [9]:
def find_peaks(pairs):
    peaks = []
    for pair in pairs:
        theta_i = pair[0][1]
        theta_j = pair[1][1]
        
        rho_i = pair[0][0]
        rho_j = pair[1][0]
        
        delta_theta = np.abs(theta_i - theta_j)
        delta_rho = np.abs(rho_i + rho_j)
        
        alpha_k = 0.5 * (theta_i + theta_j)
        xi_k = 0.5 * np.abs(rho_i + rho_j)
        xi_k_org = 0.5 * np.abs(rho_i - rho_j)
        peaks.append([xi_k, alpha_k, delta_rho, delta_theta,xi_k_org])
    return peaks

In [10]:
def detect_rectangle(horizontal_peaks, vertical_peaks, threshold, a=1,b=4):
    rectangle = []
    error = 999999
    for h in range(len(horizontal_peaks)):
        for v in range(len(vertical_peaks)):
            alpha = np.abs(np.abs(horizontal_peaks[h][1] - vertical_peaks[v][1]) - np.deg2rad(90))
            if  alpha < np.deg2rad(threshold):
                error_current =  np.sqrt((a * (np.square(horizontal_peaks[h][3]) + np.square(vertical_peaks[v][3]) + np.square(alpha))) + \
                (b * (np.square(horizontal_peaks[h][2]) + np.square(vertical_peaks[v][2]))))
                if error_current < error:
                    rectangle =[horizontal_peaks[h][0], horizontal_peaks[h][1], vertical_peaks[v][0], vertical_peaks[v][1], horizontal_peaks[h][4], vertical_peaks[v][4]]
    return rectangle

In [11]:
def find_intersection(line1, line2):
    x1, y1 = line1[0]
    x2, y2 = line1[1]
    x3, y3 = line2[0]
    x4, y4 = line2[1]
    
    # we know lines are not parallel so denominator is never 0
    denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
    
    # calculate intersection point
    intersection_x = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denominator
    intersection_y = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denominator
    
    return intersection_x, intersection_y

In [12]:
def draw_rectangle(image, center, orientation, width, height, color=(0, 255, 0), thickness=2):
    # calculate the vertices of the rectangle
    angle_rad = np.deg2rad(orientation)
    cos_angle = np.cos(angle_rad)
    sin_angle = np.sin(angle_rad)

    half_width = width / 2
    half_height = height / 2

    # calculate the corners of the rectangle
    corner1 = (int(center[0] - half_width * cos_angle - half_height * sin_angle),
               int(center[1] - half_width * sin_angle + half_height * cos_angle))
    corner2 = (int(center[0] + half_width * cos_angle - half_height * sin_angle),
               int(center[1] + half_width * sin_angle + half_height * cos_angle))
    corner3 = (int(center[0] + half_width * cos_angle + half_height * sin_angle),
               int(center[1] + half_width * sin_angle - half_height * cos_angle))
    corner4 = (int(center[0] - half_width * cos_angle + half_height * sin_angle),
               int(center[1] - half_width * sin_angle - half_height * cos_angle))

    # Draw the rectangle
    cv2.line(image, corner1, corner2, color, thickness)
    cv2.line(image, corner2, corner3, color, thickness)
    cv2.line(image, corner3, corner4, color, thickness)
    cv2.line(image, corner4, corner1, color, thickness)

In [13]:
def segment_image(image):
    segmented_image = image.copy()
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  
    thresh = threshold_otsu(gray_image)
    binary_mask = gray_image > thresh
    segments = slic(image, n_segments=50, compactness=10)
    segmentation_map = mark_boundaries(segmented_image, segments)
    return segmentation_map

### Code

In [14]:
def read_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        if os.path.isfile(img_path):
            # Read the image using OpenCV
            img = cv2.imread(img_path)
            if img is not None:
                images.append(img)
            else:
                print("Unable to read image:", img_path)
    return images

In [15]:
def detect_plate(image_original, method = "canny"):
    result_image = image_original.copy()
    # transforming to grayscale for computations
    image = cv2.cvtColor(image_original, cv2.COLOR_BGR2GRAY)  
    if method == "canny":
        edge_image = canny_edge_detection(image)
        # find hough lines
        horizontal_lines_1 = hough_lines(edge_image,edge_threshold=200, line_threshold_min=80,theta_min=0, theta_max=20,num_of_lines = 8)
        horizontal_lines_2 = hough_lines(edge_image,edge_threshold=200, line_threshold_min=80,theta_min=150, theta_max=180,num_of_lines = 8)
        horizontal_lines = horizontal_lines_1 + horizontal_lines_2
        vertical_lines = hough_lines(edge_image,edge_threshold=90, line_threshold_min=25, theta_min = 80, theta_max = 100,num_of_lines = 15)
    elif method == "sobel":
        edge_image = sobel_filter(image)[0]
        # find hough lines
        horizontal_lines_1 = hough_lines(edge_image,edge_threshold=100, line_threshold_min=80,theta_min=0, theta_max=20,num_of_lines = 15)
        horizontal_lines_2 = hough_lines(edge_image,edge_threshold=100, line_threshold_min=80,theta_min=150, theta_max=180,num_of_lines = 15)
        horizontal_lines = horizontal_lines_1 + horizontal_lines_2
        vertical_lines = hough_lines(edge_image,edge_threshold=60, line_threshold_min=25, theta_min = 80, theta_max = 100,num_of_lines = 15)
        
    # draw lines to hough image
    hough_image = image.copy()
    hough_image = draw_hough_lines(horizontal_lines, hough_image)
    hough_image = draw_hough_lines(vertical_lines, hough_image)
    # find pairs from lines
    vertical_pairs = find_pairs(vertical_lines)
    horizontal_pairs = find_pairs(horizontal_lines)
    # find peaks from pairs
    vertical_peaks = find_peaks(vertical_pairs)
    horizontal_peaks = find_peaks(horizontal_pairs)
    # find rectangle from peaks
    rectangle = detect_rectangle(horizontal_peaks, vertical_peaks, 8)
    # find the center of rectangle if there's a rectangle
    if len(rectangle) == 6:
        pt1h, pt2h = hough_to_cartesian(rectangle[0], rectangle[1])
        pt1v, pt2v = hough_to_cartesian(rectangle[2], rectangle[3])
        line1 = (pt1h, pt2h)
        line2 = (pt1v, pt2v)
        center = find_intersection(line1, line2) 
        orientation = rectangle[1]  
        width = 2 * rectangle[5]
        height = 2 * rectangle[4]  
        # draw the rectangle on the image
        draw_rectangle(result_image, center, orientation, width, height)
    return result_image, edge_image, hough_image

In [16]:
def visualize(original_image):
    segmented_image = segment_image(original_image)
    result_image_sobel, edge_image_sobel, hough_image_sobel = detect_plate(original_image,method="sobel")
    result_image_canny, edge_image_canny, hough_image_canny = detect_plate(original_image,method="canny")
    fig, axes = plt.subplots(2, 4, figsize=(16, 4))
    ax = axes.ravel()
    ax[0].imshow(original_image)
    ax[0].set_title("Original Image")
    ax[1].imshow(edge_image_sobel,cmap='gray')
    ax[1].set_title("Edge Image Sobel")
    ax[2].imshow(hough_image_sobel, cmap='gray')
    ax[2].set_title("Hough Lines Sobel")
    ax[3].imshow(result_image_sobel)
    ax[3].set_title("Result Image Sobel")
    ax[4].imshow(segmented_image)
    ax[4].set_title("Segmentation Map ")
    ax[5].imshow(edge_image_canny, cmap='gray')
    ax[5].set_title("Edge Image Canny")
    ax[6].imshow(hough_image_canny, cmap='gray')
    ax[6].set_title("Hough Lines Canny")
    ax[7].imshow(result_image_canny)
    ax[7].set_title("Result Image Canny")
    plt.tight_layout()
    plt.show()

In [17]:
folder_path = "dataset"
images = read_images_from_folder(folder_path)

In [ ]:
for image in images:
    visualize(image)

## Sobel vs Canny
The Sobel algorithm tends to blur edges, whereas the Canny algorithm generates a clear edge map. Consequently, when using Sobel, we typically need to set a much lower edge threshold for Hough transform to detect lines effectively. However, Sobel tends to introduce more nuanced edges, leading to increased noise that can hinder Hough's performance.

Conversely, Canny blurs the image prior to edge detection, resulting in sharper edges. This makes it easier for the Hough algorithm to detect lines from well-defined edges. However, this approach encounters challenges when dealing with car structures, particularly the front part, which often consists of numerous lines. Consequently, the lines defining the number plate may not always be adequately detected for reliable rectangle detection.

Overall, while Canny provides clearer edges for Hough transform, it may struggle with complex structures like cars due to the multitude of lines involved, especially in the front area.


## The performance of plate detection

In general, relying solely on the raw Hough implementation with a square algorithm proves insufficient for plate detection. Factors such as the presence of other squares in the vicinity or the small size of the plate relative to the overall image pose significant challenges. Moreover, the dataset comprises plates captured at various angles and with different designs, making it impractical to develop a single model that performs optimally across all scenarios.

Although I could enhance the algorithm by incorporating features like color detection or implementing a windowing approach to focus specifically on the plate area, I refrained from doing so as the assignment aimed to evaluate the efficacy of the Hough transform alone.

Recognizing the limitations of the Hough method for plate detection, further refinement or the adoption of additional techniques may be necessary to achieve satisfactory results, especially given the diverse nature of the dataset and the inherent complexities of the task.

In [ ]:
# if want to save the images use this function
def save_image(original_image, file_name):
    result_image_canny, edge_image_canny, hough_image_canny = detect_plate(original_image,method="canny")
    filename = f"{file_name}.png"
    cv2.imwrite(filename, result_image_canny)

I have achieved successful results with the algorithm, particularly when the image contains only a car plate. In cases where the angle or orientation varies, I've observed that with some adjustments to the threshold, the algorithm is still able to accurately detect the plate.

In [ ]:
IMAGE_PATH = "Cars33.png"
image_original = cv2.imread(IMAGE_PATH)
file_name = "Cars33_result.png"
save_image(image_original,file_name)

In [ ]:
IMAGE_PATH = "Cars102.png"
image_original = cv2.imread(IMAGE_PATH)
file_name = "Cars102_result.png"
save_image(image_original,file_name)